In [ ]:
import pandas as pd
import numpy as np
import math 

In [ ]:
data=pd.read_csv('sample_data/1.csv')
data_train=data[:int(data.shape[0]*(9/10))]
data_test=data[int(data.shape[0]*(9/10)):]

#Tree from scratch

In [ ]:
def getEntropy(d):
  ent=0
  for i in d['class'].value_counts():
    ent-=i*np.log2(i/d.shape[0])/d.shape[0]
  return ent

In [ ]:
def gain(d,f):
  rel_cap_shape_gain=getEntropy(d)
  vc=d[f].value_counts();
  for i in d[f].value_counts().index:
    rel_cap_shape_gain-=vc[i]*getEntropy(d[d[f]==i])/d.shape[0]
  return rel_cap_shape_gain

In [ ]:
class Node:
  def __init__(self):
    self.fromFeatureVal=[]
    self.name=None
    self.child=[]

In [ ]:
def buildTree(root,d,depth):
    if d['class'].value_counts().shape[0]==1:
        root.name='leaf'+' '+list(d['class'].value_counts().index)[0]
        return
    if(depth==0):
        ec= d['class'].value_counts()[0]
        pc= d['class'].value_counts()[1]
        if(ec>=pc):
            root.name='leaf e'
        else:
            root.name='leaf p'
        return
    maxGain=-100000000
    for i in d.columns:
        if(i!='class' and gain(d,i)>maxGain):
            maxGain=gain(d,i)
            feature=i
    root.name=feature
    for i in d[feature].value_counts().index:
        r=Node()
        r.fromFeatureVal.append(i)
        buildTree(r,d[d[feature]==i].drop(columns=[feature]),depth-1)
        root.child.append(r)
    #Combining leave nodes
    e=Node()
    e.name='leaf e'
    p=Node()
    p.name='leaf p'
    iter=root.child.copy()
    for i in iter:
        if i.name=='leaf e':
            e.fromFeatureVal.append(i.fromFeatureVal[0])
            root.child.remove(i)
        if i.name=='leaf p':
            p.fromFeatureVal.append(i.fromFeatureVal[0])
            root.child.remove(i)
    if(len(e.fromFeatureVal)!=0): 
        root.child.append(e)
    if(len(p.fromFeatureVal)!=0): 
        root.child.append(p)
    return


In [ ]:
def rep(root,s):
  print(s,root.fromFeatureVal," ",root.name)
  for i in root.child:
    rep(i,s+"     ")

In [ ]:
def predict(root,t1):
    if 'leaf' in root.name:
        return root.name
    for i in root.child:
        if list(t1[root.name])[0] in i.fromFeatureVal:
            return predict(i,t1)

In [ ]:
root=Node()
root.fromFeatureVal.append('root')
buildTree(root,data_train,10)
rep(root,"")

 ['root']   odor
      ['n']   spore-print-color
           ['w']   habitat
                ['l']   cap-color
                     ['c', 'n']   leaf e
                     ['w', 'y']   leaf p
                ['d']   gill-size
                     ['b']   leaf e
                     ['n']   leaf p
                ['w', 'g', 'p']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p


In [ ]:
pred=[]
for i in data_test.index:
    t1=data_test.loc[i,:].to_frame().T
    pred.append(predict(root , t1))
correct=0
for i,j in zip(pred,data_test['class']):
    if isinstance(i,str):
        if j in i:
            correct+=1
print(correct, " out of ",data_test.shape[0]," correct")
print("Acuuracy: ",correct/data_test.shape[0])

813  out of  813  correct
Acuuracy:  1.0


##Pre-pruning using max-depth

###finding best depth using n validation set 

In [ ]:
from sklearn.model_selection import train_test_split
from statistics import mean

In [ ]:
def find_accuracy(data_test,root):
  pred=[]
  for i in data_test.index:
      t1=data_test.loc[i,:].to_frame().T
      pred.append(predict(root , t1))
  correct=0
  for i,j in zip(pred,data_test['class']):
      if isinstance(i,str):
          if j in i:
              correct+=1
  #print(correct, " out of ",data_test.shape[0]," correct")
  #print("Acuuracy: ",correct/data_test.shape[0])
  return correct/data_test.shape[0]


In [ ]:
def print_trees(trees):
  for i in len(trees):
    print('The following are trees formed by validation')

In [ ]:
depth,trees1,accuracy,avg_acc=[1,2,3,4,5],[],[],[]
n=3
for i in depth:
  trees1.append([])
  accuracy.append([])
  for j in range(0,n):
    data_train_train, data_train_val=train_test_split(data_train,test_size=0.1)
    root=Node()
    root.fromFeatureVal.append('root')
    buildTree(root,data_train_train,i)
    trees1[depth.index(i)].append(root)
    accuracy[depth.index(i)].append(find_accuracy(data_train_val,root))
  avg_acc.append(mean(accuracy[depth.index(i)]))

####Finding best accuracy

In [ ]:
avg_acc

[0.98224043715847, 0.99408014571949, 0.99408014571949, 1.0, 1.0]

In [ ]:
j=1
for i in trees1:
  print("Depth: ",j)
  j+=1
  rep(i[0],"")

Depth:  1
 ['root']   odor
      ['n', 'l', 'a']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
Depth:  2
 ['root']   odor
      ['n']   spore-print-color
           ['n', 'k', 'w', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
Depth:  3
 ['root']   odor
      ['n']   spore-print-color
           ['w']   habitat
                ['w', 'g', 'l', 'd', 'p']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
Depth:  4
 ['root']   odor
      ['n']   spore-print-color
           ['w']   habitat
                ['l']   cap-color
                     ['c', 'n']   leaf e
                     ['w', 'y']   leaf p
                ['d']   gill-size
                     ['b']   leaf e
                     ['n']   leaf p
                ['w', 'g', 'p']   leaf e
           ['k', 'n', 'h', 'o', 'y', 'b']   leaf

In [ ]:
def best_depth(avg_acc):
  max_acc, bd=0,1
  for i in range(len(avg_acc)):
    if avg_acc[i]<0.999 and avg_acc[i]>max_acc:
      max_acc=avg_acc[i]
      bd=depth[i]
  return bd

In [ ]:
best_depth(avg_acc)

2

In [ ]:
rep(trees1[1][2],"")

 ['root']   odor
      ['n']   spore-print-color
           ['k', 'n', 'w', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['l', 'a']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p


##Post-pruning-max depth 

In [ ]:
root=Node()
root.fromFeatureVal.append('root')
buildTree(root,data_train,10)
rep(root,"")

 ['root']   odor
      ['n']   spore-print-color
           ['w']   habitat
                ['l']   cap-color
                     ['c', 'n']   leaf e
                     ['w', 'y']   leaf p
                ['d']   gill-size
                     ['b']   leaf e
                     ['n']   leaf p
                ['w', 'g', 'p']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p


In [ ]:
find_accuracy(data_test,root)

1.0

####helper function to find leaf value

In [ ]:
def find_cnt(root,cl):
  if(root.name=='leaf p'):
    rl=[cl[0]+1,cl[1]]
    return rl
  if(root.name=='leaf e'):
    rl=[cl[0],cl[1]+1]
    return rl
  rl=[0,0]
  for i in root.child:
    tl=[0,0]
    cc=find_cnt(i,tl)
    rl[0]+=cc[0]
    rl[1]+=cc[1]
  return rl

###main post-pruning function

In [ ]:
import random

In [ ]:
def post_prune(root,depth):
  if(root.name=='leaf p' or root.name=='leaf e'):
    return root
  if(depth==0):
    cl=[0,0]
    class_cnt=find_cnt(root,cl)
    if(class_cnt[0]>class_cnt[1]): 
      root.name='leaf p'
      root.child.clear()
    else:
      root.name='leaf e'
      root.child.clear()
    return root
  i= root.child[0]
  root.child[root.child.index(i)]=post_prune(i,depth-1)
  return root

In [ ]:
import copy

In [ ]:
depth,trees2,accuracy,avg_acc=[1,2,3],[],[],[]
n=3
for i in depth:
  trees2.append([])
  accuracy.append([])
  for j in range(0,n):
    data_train_train, data_train_val=train_test_split(data_train,test_size=0.1)
    root=Node()
    root.fromFeatureVal.append('root')
    buildTree(root,data_train,10)
    test_root=copy.copy(root)
    test_root=post_prune(test_root,i)
    trees2[depth.index(i)].append(test_root)
    accuracy[depth.index(i)].append(find_accuracy(data_train_val,test_root))
  avg_acc.append(mean(accuracy[depth.index(i)]))

In [ ]:
avg_acc

[0.9854280510018215, 0.9931693989071039, 0.9990892531876139]

In [ ]:
def best_depth2(avg_acc):
  max_acc, bd=0,1
  for i in range(len(avg_acc)):
    if avg_acc[i]<0.995 and avg_acc[i]>max_acc:
      max_acc=avg_acc[i]
      bd=depth[i]
  return bd

In [ ]:
j=1
for i in trees2:
  print("Depth: ",j)
  j+=1
  rep(i[0],"")

Depth:  1
 ['root']   odor
      ['n']   leaf e
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
Depth:  2
 ['root']   odor
      ['n']   spore-print-color
           ['w']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
Depth:  3
 ['root']   odor
      ['n']   spore-print-color
           ['w']   habitat
                ['l']   leaf e
                ['d']   gill-size
                     ['b']   leaf e
                     ['n']   leaf p
                ['w', 'g', 'p']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p


In [ ]:
best_depth2(avg_acc)

2

In [ ]:
rep(trees2[1][0],"")

 ['root']   odor
      ['n']   spore-print-color
           ['w']   leaf e
           ['n', 'k', 'h', 'o', 'y', 'b']   leaf e
           ['r']   leaf p
      ['a', 'l']   leaf e
      ['f', 'y', 's', 'p', 'c', 'm']   leaf p
